In [36]:
import torch
import torch.nn as nn
import torch._C as _C

import functools
import warnings
from typing import Any
from collections import OrderedDict

# import torch.utils.hooks as hooks
# from torch.utils.hooks import RemovableHandle
from collections import OrderedDict
import weakref
import warnings

class RemovableHandle:
    
    next_id = 0
    
    def __init__(self, hooks_dict):
        self.hooks_

In [45]:
from torch import Tensor, device, dtype
from typing import Callable, Union, Tuple


_grad_t = Union[Tuple[Tensor, ...], Tensor]

def register_hook(
    self,
    hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]
) -> RemovableHandle:
    """
    psuedo code for Module's register hook_fn
    _hooks는 아래 세 가지 중 하나
        ``_forward_pre_hooks``
        ``_forward_hooks``
        ``_backward_hooks``
    attr: ``hook``은 아래와 같은 함수
        # 아래 함수를 정의하여 gradient를 쉽게 다룰 수 있음
        # 코드가 돌아가며 gradient가 어떻게 변화하는지 디버깅하거나
        # 중간에 결과값을 반환하여 다른 작업 등을 할 수 있음
        def hook_fn(module, grad_input, grad_output):
            print(grad_input)
            print(grad_output)
            return (grad_input, grad_output)
    hook 등록 메서드가 작동하는 방식은 단순히 추가하는 방식
    """
    handle = RemovableHandle(self._hooks)
    self.hooks[handle.id] = hook
    return handle

NameError: name 'RemovableHandle' is not defined

In [6]:
def with_methclass(meta, *bases):
    """Create a base class with a metaclass"""
    class metaclass(meta):
        def __new__(cls, name, this_bases, d):
            return meta(name, bases, d)
    return type.__new__(metaclass, 'temporary_class', (), {})

In [35]:
_C._FunctionBase.__dict__.keys()

dict_keys(['__new__', 'apply', '_do_backward', '_register_hook_dict', 'register_hook', 'saved_tensors', 'saved_variables', 'next_functions', 'to_save', 'non_differentiable', 'dirty_tensors', 'needs_input_grad', 'requires_grad', 'metadata', '__doc__'])

In [38]:
class _ContextMethodMixin:
    
    def save_for_backward(self, *tensors):
        """
        :func:`~Function.backward`를 위해 주어진 tensor를 저장
        
        이 메서드는 :func:`forward` 메서드 내부에서 한 번만 호출돼야 함
        
        이후 저장된 tensor는 :attr:`saved_tensors` 속성으로 접근 가능
        위 tensor가 user에게 반환되기 전, in-place 연산이 있나 check
        인자는 ``None``이 되도 상관 없음.        
        """
        self.to_save = tensors
        
    def make_dirty(self, *args):
        """
        in-place 연산이 수행된 tensor를 mark
        
        이 메서드는 :func:`forward` 메서드 내부에서 한 번만 호출돼야 함
        그리고 모든 인자는 inputs이 돼야 함
        
        :func:`forward` 호출에서 수정된 모든 tensor는 
        check 연산의 정확성을 보증하기 위해 이 함수에 주어짐
        함수가 호출 되기 전, 된 후에 수정됐는지 여부는 관계 X        
        """
        self.dirty_tensors = args
        
    def mark_shared_storage(self, *pairs):
        warnings.warn(
            'mark_shared_storage is deprecated. '
            'Tensors with shared storages are automatically tracked. Note '
            'that calls to `set_()` are not tracked')
    
    def mark_non_differentiable(self, *args):
        """
        output들을 non-differentiable로 표시
        
        이 메서드는 :func:`forward` 메서드 내부에서 한 번만 호출돼야 함
        그리고 모든 인자는 outputs이 돼야 함
        
        outputs에 gradient가 필요하지 않다고 표시함 (backward 연산의 효율성을 위해서!)
        :meth:`~Function.backward`의 각 출력에 gradient를 허용해야 하지만,
        이는 항상 출력의 shape과 동일한 shape의 zero tensor가 됨
        
        This is used e.g. for indices returned from a max :class:`Function`.(해석불가)
        """
        self.non_differentiable = args

In [46]:
class _HookMixin:
    
    """
    `torch.nn.Module`에도 있었던 method
    만일 backward_hooks이 없으면 OrderedDict로 handler
    아래 메서드는 일전에 본 객체와 동일
    """
    
    @staticmethod
    def _register_hook(backward_hooks, hook):
        if backward_hooks is None:
            backward_hooks = OrderedDict()
#         handle = hooks.RemovableHandle(backward_hooks)
        handle = RemovableHandle(backward_hooks)
        backward_hooks[handle.id] = hook
        return backward_hooks, handle

In [26]:
class BackwardCFunction(
    _C._FunctionBase,
    _ContextMethodMixin,
    
):


class FunctionMeta(type):
    
    def __init__(cls, name, bases, attrs):
        for super_cls in cls.mro():
            forward = super_cls.__dict__.get("forward")
            if forward is not None:
                has_static_forward = isinstance(forward, staticmethod)
                break
        cls._is_legacy = not has_static_forward
        
        # old-style functions
        # forward 메서드가 staticmethod로 정의
        if not has_static_forward:
            return super(FunctionMeta, cls).__init__(name, bases, attrs)
        
        backward_fn = type(
            name + 'Backward',     # name
            (BackwardCFunction,),  # bases
            {'_forward_cls': cls}  # attrs
        )
        cls._backward_cls = backward_fn
        
        return super(FunctionMeta, cls).__init__(name, bases, attrs)

In [5]:
class Function():
    
    def __call__(self, *args, **kwargs):
        raise RuntimeError(
            "Legacy autograd function with non-static forward method is deprecated. "
            "Please use new-style autograd function with static forward method. "
            "(Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)")
    
    # for the tensor
    is_traceable = False
    
    @staticmethod
    def forward(ctx: Any, *args: Any, **kwargs: Any) -> Any:
        raise NotImplementedError(
            "You must implement the forward function for custom"
            " autograd.Function.")
    
    @staticmethod
    def backward(ctx: Any, *grad_output: Any) -> Any:
        raise NotImplementedError(
            "You must implement the backward function for custom"
            " autograd.Function.")

In [ ]:
with_metaclass(FunctionMeta, _C._FunctionBase, _ContextMethodMixin, _HookMixin)